In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPool1D, Flatten, Dropout,LSTM

In [ ]:
# estimator = [('RF', RandomForestClassifier(n_estimators = 200, max_depth = 5)), ('GB', GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 50)),
#              ('CAT', CatBoostClassifier(depth= 5, iterations = 35, learning_rate = 0.35)), ('ADB', AdaBoostClassifier(n_estimators = 200, learning_rate = 0.1, random_state = 50))]
# Stacking = StackingClassifier( estimators=estimator, final_estimator= CatBoostClassifier(depth= 5, iterations = 35, learning_rate = 0.35))

**Train**

In [ ]:
df = pd.read_csv('/content/FastText+GDPC.csv')
columns = df.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
X = df[columns]
Y = df[target]

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.3)

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 200, max_depth = 5),
          XGBClassifier(n_estimators = 200,max_depth = 5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 5,random_state = 50),
          GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 50),
          AdaBoostClassifier(n_estimators = 200, learning_rate = 0.1, random_state = 50)]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics((FT+GDPC)-CV).csv")
# clf = StackingClassifier( estimators=estimator, final_estimator=RandomForestClassifier(n_estimators = 200, max_depth = 5))
# prob = clf.fit_transform(xtrain, ytrain)
# pd.DataFrame(prob).to_csv("total_Metics_Probability.csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=5, n_estimato...  0.833333  0.666595   
1  XGBClassifier(base_score=None, booster=None, c...  0.843137  0.686221   
2       LGBMClassifier(max_depth=5, random_state=50)  0.826797  0.653520   
3  GradientBoostingClassifier(learning_rate=0.5, ...  0.849673  0.699432   
4  AdaBoostClassifier(learning_rate=0.1, n_estima...  0.820261  0.640492   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.666581   0.833333  0.827815  0.830565     0.838710     0.827815  
1  0.686221   0.841060  0.841060  0.841060     0.845161     0.841060  
2  0.653506   0.826667  0.821192  0.823920     0.832258     0.821192  
3  0.699192   0.857143  0.834437  0.845638     0.864516     0.834437  
4  0.640369   0.824324  0.807947  0.816054     0.832258     0.807947  


**Test**

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

# cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 200, max_depth = 5),
          XGBClassifier(n_estimators = 200,max_depth = 5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 5,random_state = 50),
          GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 50),
          AdaBoostClassifier(n_estimators = 200, learning_rate = 0.1, random_state = 50)]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  model.fit(xtrain, ytrain)
  pred = model.predict(xtest)
  # pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytest, pred)
  mcc = matthews_corrcoef(ytest, pred)
  cm1 = confusion_matrix(ytest, pred)
  kappa = cohen_kappa_score(ytest, pred)
  f1 = f1_score(ytest, pred)
  precision_score = precision_score(ytest, pred)
  recall_score = recall_score(ytest, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics((FT+GDPC)-TS)).csv")
# clf = StackingClassifier( estimators=estimator, final_estimator=RandomForestClassifier(n_estimators = 200, max_depth = 5))
# prob = clf.fit_transform(xtest, ytest)
# pd.DataFrame(prob).to_csv("total_Metics_Probability(LSA+PAAC).csv")

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 151, number of negative: 155
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000876 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13884
[LightGBM] [Info] Number of data points in the train set: 306, number of used features: 153
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.493464 -> initscore=-0.026145
[LightGBM] [Info] Start training from score -0.026145
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

# **CNN**

In [ ]:
df = pd.read_csv('/content/Res-FastText+GDPC.csv')
columns = df.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
X = df[columns]
Y = df[target]

**Train**

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.3)

In [ ]:
xtrain = xtrain.to_numpy()
ytrain = ytrain.to_numpy()
xtrain = xtrain.reshape(xtrain.shape[0], xtrain.shape[1], 1)
# xtest = xtest.reshape(xtest.shape[0], xtest.shape[1], 1)

In [ ]:
kf = KFold(n_splits=5, shuffle=True)
for train_index, val_index in kf.split(xtrain):
    X_train, X_val = xtrain[train_index], xtrain[val_index]
    y_train, y_val = ytrain[train_index], ytrain[val_index]

In [ ]:
cnn = Sequential()

In [ ]:
cnn.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)))
cnn.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
cnn.add(Conv1D(filters=128, kernel_size=2, activation='relu'))

In [ ]:
cnn.add(MaxPool1D(pool_size=2))

In [ ]:
cnn.add(Flatten())

In [ ]:
cnn.add(Dense(128, activation='relu'))
cnn.add(Dense(64, activation='relu'))
cnn.add(Dense(1, activation='sigmoid'))

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
cnn.fit(X_train, y_train, epochs = 40, batch_size= 64)

Epoch 1/40
4/4 [==============================] - 2s 94ms/step - loss: 0.6925 - accuracy: 0.5347
Epoch 2/40
4/4 [==============================] - 0s 93ms/step - loss: 0.6927 - accuracy: 0.5224
Epoch 3/40
4/4 [==============================] - 0s 94ms/step - loss: 0.6841 - accuracy: 0.5224
Epoch 4/40
4/4 [==============================] - 0s 93ms/step - loss: 0.6728 - accuracy: 0.5673
Epoch 5/40
4/4 [==============================] - 0s 92ms/step - loss: 0.6484 - accuracy: 0.6367
Epoch 6/40
4/4 [==============================] - 0s 98ms/step - loss: 0.6133 - accuracy: 0.6857
Epoch 7/40
4/4 [==============================] - 0s 87ms/step - loss: 0.5559 - accuracy: 0.6939
Epoch 8/40
4/4 [==============================] - 0s 100ms/step - loss: 0.5207 - accuracy: 0.7265
Epoch 9/40
4/4 [==============================] - 1s 161ms/step - loss: 0.5147 - accuracy: 0.7224
Epoch 10/40
4/4 [==============================] - 1s 160ms/step - loss: 0.4997 - accuracy: 0.7224
Epoch 11/40
4/4 [=========

In [ ]:
pred = cnn.predict(X_val)
y_pred_classes = np.round(pred).astype(int)

2/2 [==============================] - 0s 16ms/step


In [ ]:
accuracy_score(y_val, y_pred_classes), f1_score(y_val, y_pred_classes), cohen_kappa_score(y_val, y_pred_classes), matthews_corrcoef(y_val, y_pred_classes)

(0.9016393442622951, 0.896551724137931, 0.8030139935414424, 0.804750708270836)

In [ ]:
cm1 = confusion_matrix(y_val, y_pred_classes)
specificity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
sensitivity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
specificity, sensitivity

(0.8787878787878788, 0.9285714285714286)

**Test**

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.3)

In [ ]:
sample_size = xtrain.shape[0] # number of samples in train set
time_steps  = xtrain.shape[1] # number of features in train set
input_dimension = 1               # each feature is represented by 1 number

In [ ]:
train_data_reshaped = xtrain.values.reshape(sample_size,time_steps,input_dimension)
n_timesteps = train_data_reshaped.shape[1]
n_features  = train_data_reshaped.shape[2]

In [ ]:
cnn = Sequential()
cnn.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)))
cnn.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
cnn.add(Conv1D(filters=128, kernel_size=2, activation='relu'))

In [ ]:
cnn.add(MaxPool1D(pool_size=4))

In [ ]:
cnn.add(Flatten())

In [ ]:
cnn.add(Dense(128, activation='relu'))
cnn.add(Dense(64, activation='relu'))
cnn.add(Dense(1, activation='sigmoid'))

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
cnn.fit(xtrain, ytrain, epochs = 40, batch_size= 64)

Epoch 1/40
5/5 [==============================] - 2s 83ms/step - loss: 0.6914 - accuracy: 0.5359
Epoch 2/40
5/5 [==============================] - 0s 89ms/step - loss: 0.6822 - accuracy: 0.5359
Epoch 3/40
5/5 [==============================] - 0s 79ms/step - loss: 0.6664 - accuracy: 0.5359
Epoch 4/40
5/5 [==============================] - 0s 87ms/step - loss: 0.6355 - accuracy: 0.6503
Epoch 5/40
5/5 [==============================] - 0s 86ms/step - loss: 0.5895 - accuracy: 0.7059
Epoch 6/40
5/5 [==============================] - 0s 83ms/step - loss: 0.5572 - accuracy: 0.7418
Epoch 7/40
5/5 [==============================] - 0s 85ms/step - loss: 0.5477 - accuracy: 0.7092
Epoch 8/40
5/5 [==============================] - 0s 89ms/step - loss: 0.5075 - accuracy: 0.7549
Epoch 9/40
5/5 [==============================] - 0s 85ms/step - loss: 0.4834 - accuracy: 0.7680
Epoch 10/40
5/5 [==============================] - 0s 91ms/step - loss: 0.4645 - accuracy: 0.7680
Epoch 11/40
5/5 [============

In [ ]:
pred = cnn.predict(xtest)
pred = (pred > 0.5)

5/5 [==============================] - 0s 11ms/step


In [ ]:
accuracy_score(ytest, pred), f1_score(ytest, pred), cohen_kappa_score(ytest, pred), matthews_corrcoef(ytest, pred)

(0.9393939393939394,
 0.9393939393939394,
 0.8788990825688073,
 0.8805147058823529)

In [ ]:
cm1 = confusion_matrix(ytest, pred)
specificity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
sensitivity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
specificity, sensitivity

(0.9117647058823529, 0.96875)